# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df =  pd.read_csv('../output_data/cities.csv')
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tahe,52.3333,124.7333,30.81,84,54,4.25,CN,1619724585
1,kapaa,22.0752,-159.3190,75.20,69,40,14.97,US,1619724586
2,east london,-33.0153,27.9116,59.00,94,20,8.05,ZA,1619723987
3,castro,-24.7911,-50.0119,66.99,49,1,7.67,BR,1619724587
4,enumclaw,47.2043,-121.9915,71.01,57,100,1.99,US,1619724587
...,...,...,...,...,...,...,...,...,...
523,volosovo,59.4333,29.4833,39.20,41,0,2.24,RU,1619724850
524,tete,-16.1564,33.5867,86.00,54,20,6.91,MZ,1619724594
525,nosivka,50.9315,31.5828,50.59,68,99,9.08,UA,1619724851
526,kenora,49.8167,-94.4337,39.20,56,90,8.05,CA,1619724851


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']
fig = gmaps.figure()
max_humidity = humidity.max().astype(float)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filtered_df = cities_df[(cities_df['Max Temp']>70)&(cities_df['Max Temp']<80)]
filtered_df = filtered_df[filtered_df['Wind Speed']<10]
filtered_df = filtered_df[filtered_df['Cloudiness']==0]
filtered_df = filtered_df.dropna()
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,saint-philippe,-21.3585,55.7679,77.00,78,0,6.91,RE,1619724619
112,karratha,-20.7377,116.8463,79.16,36,0,8.59,AU,1619724642
115,saint-pierre,-21.3393,55.4781,77.00,78,0,6.91,RE,1619724643
151,moron,-34.6534,-58.6198,75.20,41,0,9.22,AR,1619724662
157,chivilcoy,-34.8957,-60.0167,75.00,67,0,8.01,AR,1619724665
216,tigre,-34.4260,-58.5796,73.99,53,0,8.05,AR,1619724693
302,beloha,-25.1667,45.0500,73.74,74,0,7.38,MG,1619724736
352,canton,23.1167,113.2500,73.99,88,0,2.24,CN,1619724645
358,saint-joseph,-21.3667,55.6167,77.00,78,0,6.91,RE,1619724763
427,gat,31.6100,34.7642,75.99,69,0,7.45,IL,1619724799


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
data ={
    'Hotel Name': [],
    'Lat' : [],
    'Lng' : [],
    'City': [],
    'Country': []
}
for index, row in filtered_df.iterrows():
    print(f'Requesting data for {row["City"]}')
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": "hotel","radius": 5000,
        "type": "lodging","key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    try:
        data['Hotel Name'].append(response["results"][0]["name"])
        data['Lat'].append(response["results"][0]["geometry"]["location"]["lat"])
        data['Lng'].append(response["results"][0]["geometry"]["location"]["lng"])
        data['City'].append(row['City'])
        data['Country'].append(row['Country'])
        print('Done')        
    except:
        print(f'Incomplete data, Skiping ...')
hotel_df = pd.DataFrame.from_dict(data)
hotel_df

Requesting data for saint-philippe
Done
Requesting data for karratha
Done
Requesting data for saint-pierre
Done
Requesting data for moron
Done
Requesting data for chivilcoy
Done
Requesting data for tigre
Done
Requesting data for beloha
Incomplete data, Skiping ...
Requesting data for canton
Done
Requesting data for saint-joseph
Done
Requesting data for gat
Done
Requesting data for neuquen
Done
Requesting data for sao luiz gonzaga
Done
Requesting data for santa maria
Done
Requesting data for ambovombe
Done


,Hotel Name,Lat,Lng,City,Country
0,Les Embruns Du Baril,-21.369201,55.727949,saint-philippe,RE
1,Karratha International Hotel,-20.737639,116.841998,karratha,AU
2,Hôtel le Saint-Pierre,-21.341364,55.474339,saint-pierre,RE
3,Hotel Haedo,-34.644268,-58.594873,moron,AR
4,Tomaso - Hotel & Eventos Boutique,-34.861836,-60.064406,chivilcoy,AR
5,Puerto Delta Apartamentos,-34.418643,-58.577886,tigre,AR
6,White Swan Hotel,23.105743,113.242973,canton,CN
7,Gandalf Safari Camp,-21.375362,55.592266,saint-joseph,RE
8,Desert Gat,31.602743,34.764339,gat,IL
9,Hotel Huemul,-38.958652,-68.052328,neuquen,AR


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))